In [2]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from typing import Optional, Any
from supabase import create_client, Client

load_dotenv()

True

In [3]:
BUCKET_NAME = "fiches-operations"

# Initialize Supabase client
try:
    # Supabase connection
    supabase: Client = create_client(
        os.getenv("SUPABASE_URL"),
        os.getenv("SUPABASE_KEY"),
    )
except Exception as e:
    print(f"Error initializing supabase clients: {e}")

## **UPLOAD**

In [96]:
def upload_file_to_bucket(
    supabase_client,
    bucket_name: str,
    file_path: str,
    destination_path: str,
    content_type: str = "text/plain",
    cache_control: str = "3600"
):
    """
    Uploads a JSON file to a Supabase storage bucket.

    Args:
        bucket_name (str): Name of your Supabase storage bucket.
        file_path (str): Path to the local JSON file you want to upload.
        destination_path (str): Path inside the bucket (including filename). 
        supabase_url(str): Your Supabase project URL. 
        supabase_key(str): (Optional) Your Supabase anon/service role key.
        content_type:  MIME type for the upload (default: text/plain).
        cache_control: HTTP Cache-Control header value (default: "3600").
    Returns:
        The metadata returned by Supabase on success.
    """
    # Read the JSON file into bytes
    file_path_obj = Path(file_path)
    file_bytes = file_path_obj.read_bytes()

    # Upload JSON file
    try:
        response = supabase_client.storage \
            .from_(bucket_name) \
            .upload(
                path=destination_path,
                file=file_bytes,
                file_options={
                    "content-type": content_type,
                    "cache-control": cache_control,
                    "upsert": "true",
                },
            )
        return response
    except Exception as e:
        print(repr(e))

In [97]:
def upload_all_files(
    supabase_client,
    bucket_name: str,
    folder_path: str,
    fiche_code: str,
):
    files = [
        ("function_param_values_labeled.json", "application/json"),
        ("variables_mapping.json", "application/json"),
        ("variables_matching.json", "application/json"),
        ("string_function.txt", "text/plain")
    ]

    for file, ftype in files:
        response = upload_file_to_bucket(
            supabase_client,
            bucket_name=bucket_name,
            file_path=f"{folder_path}/{fiche_code}/{file}",
            destination_path=f"/{fiche_code}/{file}",
            content_type=ftype
        )
    return response

In [72]:
response = upload_all_files(
    supabase,
    BUCKET_NAME,
    folder_path="./data/fiches",
    fiche_code="AGRI-EQ-104",
)

In [104]:
path = './data/fiches/'
folders = [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]

for fiche_code in folders:
    response = upload_all_files(
        supabase,
        BUCKET_NAME,
        folder_path="./data/fiches",
        fiche_code=fiche_code,
    )

BAT-TH-153
BAT-SE-105
BAT-TH-154
AGRI-UT-102
AGRI-EQ-107
BAT-TH-109
BAT-EQ-125
AGRI-TH-113
AGRI-EQ-109
BAT-SE-104
BAT-TH-155
BAT-SE-103
AGRI-EQ-108
BAT-TH-139
BAT-EQ-124
BAT-TH-108
AGRI-UT-104
AGRI-EQ-106
AGRI-EQ-101
AGRI-UT-103
BAT-EN-102
BAR-TH-171
BAR-TH-176
BAR-TH-122
BAR-TH-125
BAR-TH-113
BAR-TH-148
BAR-TH-177
BAR-TH-170
BAT-EN-103
BAT-EN-104
BAR-TH-141
BAR-TH-112
BAR-TH-124
BAR-TH-123
BAR-TH-101
BAR-TH-139
BAR-EQ-115
BAR-TH-130
BAR-TH-137
BAR-TH-163
BAR-TH-155
BAR-SE-104
BAT-EN-111
BAR-SE-105
BAR-TH-165
BAR-TH-162
BAT-TH-112
BAT-EQ-130
BAT-TH-115
AGRI-TH-101
AGRI-TH-108
AGRI-EQ-112
BAR-EN-104
BAT-TH-141
BAT-TH-146
BAR-EN-103
BAT-TH-125
AGRI-TH-109
BAT-TH-122
BAT-EQ-131
BAT-TH-113
BAR-EN-102
BAT-TH-140
BAR-EN-105
BAT-TH-135
AGRI-EQ-104
AGRI-TH-119
AGRI-UT-101
BAT-EQ-117
AGRI-TH-110
BAT-TH-104
BAT-TH-103
AGRI-TH-117
BAT-TH-157
BAT-TH-159
BAT-TH-161
BAT-EQ-127
BAT-TH-105
BAT-EQ-129
AGRI-EQ-102
AGRI-TH-118
BAT-TH-134
AGRI-EQ-105
BAT-TH-158
BAT-TH-156
BAR-TH-117
BAR-TH-110
BAT-EN-101


## **READ**

In [7]:
def read_file_from_bucket(
    supabase_client,
    bucket_name: str,
    file_path: str,
    file_type: str = "txt",
    encoding: str = "utf-8",
) -> Any:
    # Read file from bucket
    try:
        raw = supabase_client.storage \
            .from_(bucket_name) \
            .download(file_path)
    except Exception as e:
        print(repr(e))

    if file_type == "txt":
        return raw.decode(encoding)
    elif file_type == "json":
        return json.loads(raw)

In [8]:
function_param_values_labeled = read_file_from_bucket(
    supabase,
    BUCKET_NAME,
    file_path="/AGRI-EQ-104/function_param_values_labeled.json",
    file_type="json"
)

In [9]:
function_param_values_labeled

{'Type de serre': ['Serres maraîchères', 'Serres horticoles']}